




# Spatial Filtering

## Setup

### Import packages

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow

Read imagem from google drive

In [2]:
image_name = "t1.jpg"

image_original = cv2.imread('drive/MyDrive/Colab Notebooks/data/' + image_name,1)

Utils

In [ ]:
def Grayscale(path):
  img = cv2.imread(path, 1)
  B, G, R = cv2.split(img)
  img_grayscale_pondered = 0.299*B+0.587*G+0.114*R
  img_grayscale_pondered = np.array(img_grayscale_pondered, dtype=np.uint8)
  return img_grayscale_pondered

## Filtering

### Math

In [ ]:
def Gauss_create_kernel(sigma=1, size_x=3, size_y=3): # Tem problema aqui
  x, y = np.meshgrid(np.linspace(-1,1,size_x), np.linspace(-1,1,size_y))
  calc = 1/((2*np.pi*(sigma**2)))
  exp = np.exp(-(((x**2) + (y**2))/(2*(sigma**2))))
  return exp*calc

In [ ]:
def Convolution(image, mask, isMedian=False, even=False):
  # Funciona para média e mediana
  # É possível convolução com máscara 2x2 se 'even' for true
  N = mask.shape[0] // 2 # Distancia da borda Exemplo: (Kernel 3x3) 3//2=1 ou (Kernal 5x5) 5//2=2
  image_with_border = cv2.copyMakeBorder(image, N, N, N, N, cv2.BORDER_CONSTANT, value=(0, 0, 0)) # A largura da borda é equivalente a distancia do ponto central da máscara com seu limite
  HIGHT, WIDTH = image_with_border.shape
  image_to_return = np.zeros(image.shape)
  for row in range(N, HIGHT-N): # Inicia em N para desconsiderar o inicio da borda e finaliza antes de atingir a borda
    for column in range(N, WIDTH-N):
        limite = 1
        if (even):
          limite = 0
        # Suponha Kernel 3x3, limite será 1 e portanto (row-N:row+N+limite) será 3x3
        # Suponha Kernel 2x2, limite será 0 e portanto (row-N:row+N+limite) será 2x3
        slice = image_with_border[row-N:row+N+limite, column-N:column+N+limite]
        total = 0;
        if (isMedian):
          # Máscara nesse caso apenas demarca a vizinhança
          slice = np.sort(slice.flatten()) # Ordena e torna unidimencional
          total = slice[slice.size // 2] # Total é a mediana
        else:
          total = np.sum(mask * slice).clip(0, 255) # Total é a soma dos produtos, clip garante que o máximo valor seja 255 e o mínimo 0
        image_to_return[row-N, column-N] = total
  return image_to_return